[Reference](https://towardsdatascience.com/solving-your-first-linear-program-in-python-9e3020a9ad32)

In [3]:
from scipy.optimize import linprog
import numpy as np
from cvxopt import matrix
from cvxopt import glpk


# The perfect cake recipe that I partially remember

# Equations to solve
# f + e + b + s = 700
# b -0.5s = 0
# f + e <= 450
# e + b <= 300
# -f + e + b -s <= 0

# X matrix
var_list = ['Flour', 'Eggs', 'Butter', 'Sugar']

# Inequality equations, LHS
A_ineq = [[1., 1., 0., 0.], [0., 1., 1., 0.], [-1., 1., -1., 1.]]

# Inequality equations, RHS
B_ineq = [450., 300.,0.]

# Equality equations, LHS
A_eq = [[1., 1., 1., 1.], [0., 0., 1., -0.5]]

# Equality equations, RHS
B_eq = [700., 0]

# Cost function
c = [0., 0., 1., 1.]  # construct a cost function

print('WITHOUT BOUNDS')
# pass these matrices to linprog, use the method 'interior-point'. '_ub' implies the upper-bound or
# inequality matrices and '_eq' imply the equality matrices
res_no_bounds = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq, b_eq=B_eq, method='interior-point')
print(res_no_bounds)

WITHOUT BOUNDS
     con: array([ 5.27055590e-08, -3.78719278e-11])
     fun: 249.99999998121916
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([3.39247208e-08, 6.95661413e+01, 7.24656159e+01])
  status: 0
 success: True
       x: array([302.8994746 , 147.10052537,  83.33333333, 166.66666665])


In [4]:
# these are the bounds that help the solver arrive at a solution 
f_b = [0., 300.]  # limits for flour
e_b = [0., 200.]  # limits for eggs
b_b = [0., 100.]  # limits for butter
s_b = [0., 200.]  # limits for sugar

res_bounds = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq, b_eq=B_eq, bounds=(f_b, e_b, b_b, s_b),
                     method='interior-point')
print('\nWITH BOUNDS')
print(res_bounds)


WITH BOUNDS
     con: array([ 4.53132998e-09, -3.25428573e-12])
     fun: 249.9999999983819
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([2.91322522e-09, 5.30261661e+01, 3.93856656e+01])
  status: 0
 success: True
       x: array([286.35949946, 163.64050053,  83.33333333, 166.66666667])


In [5]:
# Write a parser for results
def result_parser(res, var_list):
    solve_status = res.success
    if solve_status is True:
        solution_list = res.x
        soln = {var_list[i]: np.round(solution_list[i]) for i in range(len(var_list))}
    else:
        soln = []
    return soln

In [6]:
# formulate problem in terms of GLPK/cvxopt
c, A_ineq, B_ineq, A_eq, B_eq = matrix(c), matrix(A_ineq).T, matrix(B_ineq), matrix(A_eq).T, matrix(B_eq)

# solve the problem
status, solution = glpk.ilp(c, A_ineq, B_ineq, A_eq, B_eq)

if status=='optimal':
    print('solution found')
    print(solution)
else: 
    print(status)

solution found
[ 2.67e+02]
[ 1.83e+02]
[ 8.33e+01]
[ 1.67e+02]

